#  Entity Relation Classification  with BERT

This example is implemented inspired by the example for BERT fine tuning from GluonNLP API.

https://gluon-nlp.mxnet.io/examples/sentence_embedding/bert.html




###  Import neccesarry modules

In [2]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from Bert import data, model

### Setting up the environment

In [3]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
ctx = mx.cpu(0)

## Using the pre-trained BERT model



"The BERT model we will use is BERT
BASE trained on an uncased corpus of books and
the English Wikipedia dataset in the
GluonNLP model zoo."

### Get BERT



Load the pre-trained BERT model. 

In [4]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                             dataset_name='book_corpus_wiki_en_uncased',
                                             pretrained=True, ctx=ctx, use_pooler=True,
                                             use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

### Add the Classification layer, BERTClassifier

1. The original model was trained for sentence pair classificaton. 
2. Entity + Context pair will be fed to the model. 
3. Both entities with context will be treated as sentences and the model will try to predict their label. 
4. After BERT model is loaded, another layer for classification (the BERTClassifier)is attached to it.  
5. This works as follows: The `BERTClassifier` class encodes the entity representations by using a BERT base model then uses a `nn.Dense` layer for classification.

In [5]:
#Add the classification layer to the bert_base
bert_classifier = model.classification.BERTClassifier(bert_base, num_classes=6, dropout=0.1)

# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

## Data preprocessing for BERT


### Loading the dataset

1. Import the pickeled file of entity pair in a dataframe.
2. Transform the dataframe into a tsv(tab separated values) file
3. Read the tsv file and print some examples to have a feeling how does data look like before it is fed into model.

In [6]:
import pickle
import pandas as pd
entity_pair = pd.read_pickle('entity_pair.pkl')
entity_pair.head()
type(entity_pair)

pandas.core.frame.DataFrame

In [7]:
entity_pair.to_csv('train.tsv', sep = '\t', index=False)

In [8]:
tsv_file = io.open('train.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())

entity_text_1	entity_text_2	label

database traditional information retrieval techniques use a	histogram of keywords as the	USAGE

representation but oral communication may offer	offer additional indices such as	USAGE

a large database of tv	database of tv shows emotions and	PART_WHOLE

of a distributed message-passing infrastructure for dialogue	infrastructure for dialogue systems which all	MODEL-FEATURE



In [9]:
# Skip the first line, column names
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [0, 1, 2]
train_data = nlp.data.TSVDataset(filename='train.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)
sample_id = 0
# First entity context
print(train_data[sample_id][0])
# Second enitiy context
print(train_data[sample_id][1])
# relation type (label)
print(train_data[sample_id][2])

database traditional information retrieval techniques use a
histogram of keywords as the
USAGE


We will use
`BERTDatasetTransform` to perform the following transformations:
- tokenize
the
input sequences
- insert [CLS] at the beginning
- insert [SEP] between entities and at the end
- generate segment ids to indicate whether a token belongs to the first sequence or the second sequence.
- generate valid length for entity

In [10]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence, the averge length of the entity
max_len = 12

all_labels = ["USAGE", "PART_WHOLE", "MODEL-FEATURE", "RESULT", "COMPARE", "TOPIC"]

pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)
data_to_train = train_data.transform(transform)

In [22]:
type(data_to_train)

mxnet.gluon.data.dataset._LazyTransformDataset

1. Followings are printout of how the BERT vocabulary looks like, with the vocabulary size and special tokens. 
2. The ids of the special tokens


In [11]:
print('vocabulary used for tokenization = %s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = %s'%data_to_train[sample_id][0])
print('valid length = %s'%data_to_train[sample_id][1])
print('segment ids = %s'%data_to_train[sample_id][2])
print('label = %s'%data_to_train[sample_id][3])

vocabulary used for tokenization = Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = [    2  7809  3151  2592 26384  5461     3  2010  3406 13113  1997     3]
valid length = 12
segment ids = [0 0 0 0 0 0 0 1 1 1 1 1]
label = [0]


## Fine-tuning the model

In [12]:
# The hyperparameters
batch_size = 32
lr = 5e-6

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in data_to_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_to_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all parameters
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

In [60]:
# Training the model with different number of epochs 3, 10, 5
# The output is from the last training with 5 epohs
# Trained in 18 epoch in total
log_interval = 4
num_epochs = 5

for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0 # for outputing the loss that happens in each step. 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data 
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # Forward computation
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # Backwards computation
        ls.backward()

        # Gradient clipping
        trainer.allreduce_grads()
        nlp.utils.clip_grad_global_norm(params, 1)
        trainer.update(1)

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0

[Epoch 0 Batch 4/40] loss=0.6277, lr=0.0000050, acc=0.836
[Epoch 0 Batch 8/40] loss=0.5569, lr=0.0000050, acc=0.836
[Epoch 0 Batch 12/40] loss=0.5909, lr=0.0000050, acc=0.826
[Epoch 0 Batch 16/40] loss=0.7349, lr=0.0000050, acc=0.818
[Epoch 0 Batch 20/40] loss=0.5884, lr=0.0000050, acc=0.820
[Epoch 0 Batch 24/40] loss=0.5854, lr=0.0000050, acc=0.821
[Epoch 0 Batch 28/40] loss=0.6485, lr=0.0000050, acc=0.817
[Epoch 0 Batch 32/40] loss=0.5894, lr=0.0000050, acc=0.815
[Epoch 0 Batch 36/40] loss=0.5135, lr=0.0000050, acc=0.815
[Epoch 0 Batch 40/40] loss=0.5256, lr=0.0000050, acc=0.821
[Epoch 1 Batch 4/40] loss=0.5420, lr=0.0000050, acc=0.886
[Epoch 1 Batch 8/40] loss=0.5858, lr=0.0000050, acc=0.854
[Epoch 1 Batch 12/40] loss=0.6092, lr=0.0000050, acc=0.834
[Epoch 1 Batch 16/40] loss=0.4557, lr=0.0000050, acc=0.851
[Epoch 1 Batch 20/40] loss=0.5447, lr=0.0000050, acc=0.848
[Epoch 1 Batch 24/40] loss=0.5059, lr=0.0000050, acc=0.852
[Epoch 1 Batch 28/40] loss=0.6864, lr=0.0000050, acc=0.842
[

## Upload and Transform Testing Data
Do the same pre-processing as we did for training data. 

In [14]:
#read the test file 
test_file = io.open('test_data_with_labels.tsv', encoding='utf-8')
for i in range(5):
    print(test_file.readline())

entity_context_1	entity_context_2	label

labeling (srl) arguments are usually	in a syntax subtree it arguments	PART_WHOLE

to label arguments locally in	such a sub-tree rather than	PART_WHOLE

approach the anchor group approach achieves an	achieves an accuracy of 87.75%	RESULT

that the prediction of mp improves semantic	mp improves semantic role labeling	RESULT



In [15]:
# Skip the first line it holds the column names
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
# Fields to select from the file
field_indices = [0, 1, 2]
test_data = nlp.data.TSVDataset(filename='test_data_with_labels.tsv',
                                 field_separator=field_separator,
                                 num_discard_samples=num_discard_samples,
                                 field_indices=field_indices)

#printout an example of testing data.
sample_id = 3
# First entity context
print(test_data[sample_id][0])
# Second enitiy context
print(test_data[sample_id][1])
#label
print(test_data[sample_id][2])

that the prediction of mp improves semantic
mp improves semantic role labeling
RESULT


In [16]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence as we have specified in previous examples
max_len = 12
all_labels = ["USAGE", "PART_WHOLE", "MODEL-FEATURE", "RESULT", "COMPARE", "TOPIC"]
pair = True
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                                                class_labels=all_labels,
                                                has_label=True,
                                                pad=True,
                                                pair=pair)

test = test_data.transform(transform)

In [18]:
print('vocabulary used for tokenization = %s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = %s'%test[sample_id][0])
print('valid length = %s'%test[sample_id][1])
print('segment ids = %s'%test[sample_id][2])
print('label = %s'%test[sample_id][3])

vocabulary used for tokenization = Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = [    2  2008  1996 17547  1997  6131     3  6131 24840 21641  2535     3]
valid length = 12
segment ids = [0 0 0 0 0 0 0 1 1 1 1 1]
label = [3]


## Predicting the label

In [66]:
# Set the hyperparameters
batch_size = 1

# Take the rows from the test data one by one  
test_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[1]) for item in test],
                                           batch_size=batch_size, shuffle=False)
# Load the test data
test_dataloader = mx.gluon.data.DataLoader(test, batch_sampler=test_sampler)

In [67]:
# Make predictions of the label
probabilities = []
metric.reset()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):

    # Load the data
    token_ids = token_ids.as_in_context(ctx)
    valid_length = valid_length.as_in_context(ctx)
    segment_ids = segment_ids.as_in_context(ctx)
    label = label.as_in_context(ctx)
    
     # Forward computation
    pred = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
    probabilities.append(pred)
    metric.update([label], [pred])

    #print(metric)

In [110]:
#print(probabilities)

In [79]:
type(probabilities[0])

mxnet.ndarray.ndarray.NDArray

## Output Transformation
1. The type of the probabilities that comes as an output from the model is mxnet.ndarray.ndarray.NDArray, but we want it to be a list. 
2. We first convert the mxnet array into a numpy array. 
3. Then convert the numpy array into a list.
4. Convert the list into a dataframe and store as a pkl file for later use in model evaluations. 

In [83]:
import numpy as np 
numpy_prob = [probabilities[i].asnumpy() for i in range(len(probabilities))] 

In [91]:
#convert the numpy array into a list
predictions = [numpy_prob[i].tolist() for i in range(len(numpy_prob))] 

In [109]:
# convert predictions as a dataframe and pickle the file
bert_output = pd.DataFrame(predictions)
bert_output.to_pickle('bert_predictions.pkl')